# Project Title
### Data Engineering Capstone Project

#### Project Summary
The purpose of this project is to create a Data Lake for the analysis of Immigration Data.  The source datasets include I94 data, US Cities Demographic data, airport code data and temperature data.  The Data Lake will store the information in Parquet format and allow Data Scientists to gain key insights into immigration patterns, including what airports people are coming through, if temperature has any relation to immigration at a particular location and if immigrants from certain locations tend to gravitate twords locations with specific demographics.  This project will create a source of truth data lake and is intended to be combined with other data sources.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os, glob
import configparser
from datetime import datetime, timedelta

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import types as T
from pyspark.sql.types import StructType as R, StructField as Fld
import pyspark.sql.functions

from read_write import *
from cleaning import *
from quality_check import *



In [2]:
# Configuration and paths
config = configparser.ConfigParser()
config.read('dl.cfg')

# Ingest Paths
i94_data_path = config['INPUT_PATHS']['I94_INPUT']
worldtemp_path = config['INPUT_PATHS']['WORLDTEMP_INPUT']
demog_path = config['INPUT_PATHS']['DEMOG_INPUT']
arptcodes_path = config['INPUT_PATHS']['AIPORTCODE_INPUT']

# Output Paths
staged_path = config['OUTPUT_PATHS']['STAGED_PATH']
output_path = config['OUTPUT_PATHS']['DATALAKE_PATH']

# Lake Folders
i94_fldr = config['LAKE_FOLDER']['IMMI_FOLDER']
worldtemp_fldr = config['LAKE_FOLDER']['WRLDTMP_FOLDER']
demog_fldr = config['LAKE_FOLDER']['DEMOG_FOLDER']
arptcodes_fldr = config['LAKE_FOLDER']['ARPTCD_FOLDER']



In [3]:
# Create spark session

spark = SparkSession.builder \
        .appName("capstone") \
        .config("spark.jars.repositories", "https://repos.spark-packages.org/") \
        .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11") \
        .enableHiveSupport().getOrCreate()

spark


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

A Data Lake is being created for the analysis of immigration patterns, including city demographics and temperature.  Apache Spark is used for performing the data processing because it can handle very large datasets and is a scalable solution as datasets grow.  In the end, data scientists will be able to use the Data Lake to mine for patterns relating to immigration.  The data lake is also intended to be combined with other sources of data.  The Data Lake will be a Source of Truth.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The datasets I am using include:
- I94 Immigration data from the National Travel and Tourism Office (NTTO): [https://www.trade.gov/national-travel-and-tourism-office]
- World Temperature Data from Climate Change: Earth Surface Temperature Data: [https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data]
- US Cities Demographics Data: [https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/]
- Airport Code Data: [https://datahub.io/core/airport-codes#data]

In [4]:
# Ingest I94 Immigration Data
df_i94 = ingest_i94(spark, i94_data_path, staged_path)

Loading: /data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
Loading: /data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat


In [5]:
# Ingest World Temperature Data by City
df_worldtemp = ingest_worldtemp(spark, worldtemp_path)

In [6]:
# Ingest US Cities Demographics
df_demog = ingest_demog(spark, demog_path)

In [7]:
# Ingest Airport Code Data by City
df_arptcodes = ingest_arptcodes(spark, arptcodes_path)

#### Show Data Samples, Schemas and record counts for source data

##### Immigration Data from Source

In [8]:
print("I94 Immigration Data:")

# Using show instead of toPandas inorder to get a complete view of the columns
df_i94.show(10)


I94 Immigration Data:
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|5680949.0|2016.0|   7.0| 117.0| 117.0|    NYC|20659.0|    1.0|     NY|   null|  30.0|    3.0|  1.0|20160724|     NPL| null|      G|   null|   null|   null| 1986.0|     D/S|     F|  null|     IG|2.947450085E9| 3940|      F1|
|5680950.0|2016.0|   7.0| 245.0| 245.0|    DET|20659.0|    1.0|     IL|20679.0

In [9]:
print("I94 Immigration Data Schema:")
df_i94.printSchema()

I94 Immigration Data Schema:
root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-

In [10]:
print("I94 Immigration Data Schema Record Count: {}".format(df_i94.count()))

I94 Immigration Data Schema Record Count: 40790529


##### Global Land Temperatures from Source

In [11]:
print("Global Land Temperatures by City:")
df_worldtemp.sort("dt", ascending=False).limit(10).toPandas()

Global Land Temperatures by City:


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,2013-09-01,NaN,NaN,Abadan,Iran,29.74N,48.00E
1,2013-09-01,NaN,NaN,Abiko,Japan,36.17N,139.23E
2,2013-09-01,NaN,NaN,Abakaliki,Nigeria,5.63N,8.07E
3,2013-09-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,2013-09-01,NaN,NaN,Abakan,Russia,53.84N,91.36E
5,2013-09-01,NaN,NaN,Çorum,Turkey,40.99N,34.08E
6,2013-09-01,15.252,1.056,Abbotsford,Canada,49.03N,122.45W
7,2013-09-01,NaN,NaN,Ürümqi,China,44.20N,87.20E
8,2013-09-01,NaN,NaN,Abengourou,Côte D'Ivoire,7.23N,4.05W
9,2013-09-01,NaN,NaN,Aachen,Germany,50.63N,6.34E


In [12]:
print("Global Land Temperatures by City Schema:")
df_worldtemp.printSchema()

Global Land Temperatures by City Schema:
root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [13]:
print("Global Land Temperatures by City Record Count: {}".format(df_worldtemp.count()))

Global Land Temperatures by City Record Count: 8599212


##### US Cities Demographics Data from Source

In [14]:
print("US Cities Demographics Data:")
df_demog.limit(10).toPandas()

US Cities Demographics Data:


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229,62432,118661,6634,7517,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712,41971,80683,4815,8355,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629,56860,108489,3800,37038,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762,43270,85032,5783,3269,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751,58077,109828,5204,16315,2.65,NC,Asian,11060


In [15]:
print("US Cities Demographics Data Schema:")
df_demog.printSchema()

US Cities Demographics Data Schema:
root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



In [16]:
print("US Cities Demographics Record Count: {}".format(df_demog.count()))

US Cities Demographics Record Count: 2891


##### Airport Code Data from Source

In [17]:
print("Airport Code Data:")
df_arptcodes.limit(10).toPandas()

Airport Code Data:


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736"


In [18]:
print("Airport Code Data Schema:")
df_arptcodes.printSchema()

Airport Code Data Schema:
root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [19]:
print("Airport Code Data Schema Record Count: {}".format(df_arptcodes.count()))

Airport Code Data Schema Record Count: 55075


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

The following data quality issues have been identified:
- Some of the columns in the source data are not very descriptive and/or valid for writing Parquet files
- Some columns have data stored as double where float would be adaquate
- Some columns have a date in SAS format where a regular date would be more useful
- In the US Demographics dataset, there are some null values in numerical fields (i.e. Male Population, Female Population, etc...)
- In the Global Temperature dataset, there are some null values
- The coordinates field in the Airport Codes dataset was had an error in which longitude came before latitude (confirmed with Google Maps)
- Null values in Airport Codes dataset for elevation, not touching as converting to zero can be erroneuos and preserving the remaining data for use
- Null values in Airport Codes dataset for municipality, gps_code, iata_code, local_code.  Not converting because there is other data still available to be used and the nulls speak for themselves

#### Cleaning Steps
Document steps necessary to clean the data

I94 Immigration Data cleaning steps includes:
- Changing column names to be more descriptive
- Correcting the data types in the columns
- Changing SAS date to YYYY-MM-DD format
- Removing duplicate rows

US Demographics Data cleaning steps include:
- Change column names to be more discriptive and transform to allow writing to Parquet
- Changing null values in numerical fields to 0
- Remove duplicate rows

Global Temperature Data cleaning steps include:
- Change column names to be easier to use
- Adjust the data type for temperature average and temperature average uncertainty to be float
- Remove rows without temperature average
- Remove dublicate rows

Airport Code Data cleaning steps include:
- Split latitude and longitude from coordinates field.  Note, the coordinates field had latitude and longitude backwards; it would be easy to generate a corrected coordinates filed, however, it is best to keep values in their own field for better usability and the decision was made to split the values into their own field.

##### I94 Data Cleaning and Display

In [20]:
# Performing cleaning tasks here

# Clean I94 Immigration data
df_i94 = clean_i94(df_i94)

In [21]:
# Show cleaned I94 Data
print("I94 Immigration Data Clean:")
df_i94.show(10)

I94 Immigration Data Clean:
+------+-------+--------+---------+------------------+----------------+----------+----------+---------+-------------+---------+------------+---------+-------------+--------+--------+--------+----------+----------+-------+------------+----------+----------+---------+------------+--------------+-------------+---------------+
|gender| cic_id|i94_year|i94_month|birth_country_code|res_country_code|admit_port|trans_mode|arr_state|immigrant_age|visa_code|summary_stat|visa_post|us_occupation|arr_flag|dep_flag|upd_flag|match_flag|birth_year|ins_num|airline_code| admit_num|flight_num|visa_type|arrival_date|departure_date|file_add_date|addmission_date|
+------+-------+--------+---------+------------------+----------------+----------+----------+---------+-------------+---------+------------+---------+-------------+--------+--------+--------+----------+----------+-------+------------+----------+----------+---------+------------+--------------+-------------+--------------

In [22]:
# Show cleaned I94 Data schema
print("I94 Immigration Data Schema Clean:")
df_i94.printSchema()

I94 Immigration Data Schema Clean:
root
 |-- gender: string (nullable = true)
 |-- cic_id: integer (nullable = true)
 |-- i94_year: integer (nullable = true)
 |-- i94_month: integer (nullable = true)
 |-- birth_country_code: integer (nullable = true)
 |-- res_country_code: integer (nullable = true)
 |-- admit_port: string (nullable = true)
 |-- trans_mode: integer (nullable = true)
 |-- arr_state: string (nullable = true)
 |-- immigrant_age: integer (nullable = true)
 |-- visa_code: integer (nullable = true)
 |-- summary_stat: integer (nullable = true)
 |-- visa_post: integer (nullable = true)
 |-- us_occupation: string (nullable = true)
 |-- arr_flag: string (nullable = true)
 |-- dep_flag: string (nullable = true)
 |-- upd_flag: string (nullable = true)
 |-- match_flag: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- ins_num: string (nullable = true)
 |-- airline_code: string (nullable = true)
 |-- admit_num: integer (nullable = true)
 |-- flight_num: string 

In [23]:
print("I94 Immigration Data Record Count Clean: {}".format(df_i94.count()))

I94 Immigration Data Record Count Clean: 40790529


##### US Demographics Data Cleaning and Display

In [24]:
df_demog = clean_demog(df_demog)

In [25]:
# Show sample of clean demographics data
print("US Cities Demographics Data Clean:")
df_demog.limit(10).toPandas()

US Cities Demographics Data Clean:


,city,state,median_age,male_population,female_population,total_population,num_of_veterans,foreign_born,avg_household_size,state_code,race,count
0,Clarksville,Tennessee,29.7,75029,74161,149190,20803,8211,2.64,TN,American Indian and Alaska Native,2338
1,El Paso,Texas,33.1,332797,348339,681136,47693,159709,3.03,TX,American Indian and Alaska Native,7359
2,Miami Beach,Florida,42.5,48090,44221,92311,2265,49908,2.10,FL,Asian,2772
3,Lincoln,Nebraska,32.3,138544,138802,277346,14694,22958,2.40,NE,White,246677
4,Richmond,California,35.3,52615,57100,109715,3611,42215,2.87,CA,Black or African-American,27434
5,Dearborn,Michigan,33.5,46482,48698,95180,3163,23920,3.04,MI,Black or African-American,3699
6,Portland,Oregon,36.7,313516,318671,632187,29940,86041,2.43,OR,American Indian and Alaska Native,15314
7,Independence,Missouri,38.6,54348,62907,117255,10220,4911,2.33,MO,Asian,558
8,Lafayette,Louisiana,31.7,64018,63643,127661,5206,5396,2.54,LA,Black or African-American,39862
9,Rochester,New York,31.4,100135,109673,209808,7288,17735,2.36,NY,White,105636


In [26]:
# Show clean demographics data Schema
print("US Cities Demographics Data Schema Clean:")
df_demog.printSchema()

US Cities Demographics Data Schema Clean:
root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: double (nullable = false)
 |-- male_population: integer (nullable = false)
 |-- female_population: integer (nullable = false)
 |-- total_population: integer (nullable = false)
 |-- num_of_veterans: integer (nullable = false)
 |-- foreign_born: integer (nullable = true)
 |-- avg_household_size: double (nullable = false)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- count: integer (nullable = false)



In [27]:
print("US Cities Demographics Record Count Clean: {}".format(df_demog.count()))

US Cities Demographics Record Count Clean: 2891


##### Global Temperature Data Clean and Display

In [28]:
# Clean Global Temperature Data
df_worldtemp = clean_worldtemp(df_worldtemp)

In [29]:
print("Global Land Temperatures by City Clean:")
df_worldtemp.sort("dt", ascending=False).limit(10).toPandas()

Global Land Temperatures by City Clean:


,city,country,latitude,longitude,date_stamp,avg_temp,avg_temp_uncert
0,Arvada,United States,39.38N,106.13W,2013-09-01,12.288000,0.911
1,Westminster,United States,39.38N,104.05W,2013-09-01,19.372999,1.054
2,Morelia,Mexico,20.09N,100.66W,2013-09-01,19.716000,1.208
3,Columbus,United States,39.38N,83.24W,2013-09-01,19.681000,1.090
4,Norfolk,United States,36.17N,75.58W,2013-09-01,22.937000,1.053
5,Yonkers,United States,40.99N,74.56W,2013-09-01,17.408001,1.048
6,Nueva San Salvador,El Salvador,13.66N,90.00W,2013-09-01,27.162001,1.286
7,San Juan Sacatepéquez,Guatemala,15.27N,90.83W,2013-09-01,21.815001,1.233
8,Apodaca,Mexico,26.52N,100.30W,2013-09-01,27.934000,1.252
9,Guaymas,Mexico,28.13N,110.00W,2013-09-01,30.885000,0.965


In [30]:
print("Global Land Temperatures by City Schema Clean:")
df_worldtemp.printSchema()

Global Land Temperatures by City Schema Clean:
root
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- date_stamp: date (nullable = true)
 |-- avg_temp: float (nullable = true)
 |-- avg_temp_uncert: float (nullable = true)



In [31]:
print("Global Land Temperatures by City Record Count Clean: {}".format(df_worldtemp.count()))

Global Land Temperatures by City Record Count Clean: 8235082


##### Airport Code Data Clean and Display

In [32]:
df_arptcodes = clean_arptcodes(df_arptcodes)

In [33]:
print("Airport Code Data Clean:")
df_arptcodes.limit(10).toPandas()

Airport Code Data Clean:


,airport_id,airport_type,airport_name,elevation_ft,continent_abbrv,country_code,region_code,municipality,gps_code,iata_code,local_code,latitude,longitude
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,40.070801,-74.933601
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,38.704022,-101.473911
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,59.949200,-151.695999
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,34.864799,-86.770302
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,35.608700,-91.254898
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,34.942803,-97.818019
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,34.305599,-112.165001
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,35.350498,-116.888000
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,39.427188,-121.763427
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,32.727374,-116.459742


In [34]:
print("Airport Code Data Schema Clean:")
df_arptcodes.printSchema()

Airport Code Data Schema Clean:
root
 |-- airport_id: string (nullable = true)
 |-- airport_type: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent_abbrv: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [35]:
print("Airport Code Data Schema Record Count Clean: {}".format(df_arptcodes.count()))

Airport Code Data Schema Record Count Clean: 55075


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

I chose an unstructured yet flexible format.  As my target users are Data Scientists, they need to be able to combine data from any source to the data in the data lake.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Extract the data from the source
2. Assess the data for cleaning needing to be done
3. Clean the data by correcting column names, adjusting data types, removing duplicates and replacing invalid values
4. Write the cleaned data to it's cooresponding destination in the Data Lake
5. As dataframes are no longer needed, do garbage collection and free up space

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [36]:
# Write I94 Immigration Data Fact

write_i94_fact(df_i94, output_path, i94_fldr)

In [37]:
# Write World Temperature by City Data Dimension

write_worldtemp_dim(df_worldtemp, output_path, worldtemp_fldr)

In [38]:
# Write US City Demographics Dimension

write_demog_dim(df_demog, output_path, demog_fldr)

In [39]:
# Write Airport Codes Dimension

write_arptcodes_dim(df_arptcodes, output_path, arptcodes_fldr)

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
##### Data Quality Checks performed:

- Check that data is loaded into the tables
- Check that there are no nulls in Primary Key fields
 
Run Quality Checks

In [4]:
# Perform quality checks here

# Load Data from data lake
df_i94 = read_lake(spark, output_path, i94_fldr)
df_worldtemp = read_lake(spark, output_path, worldtemp_fldr)
df_demog = read_lake(spark, output_path, demog_fldr)
df_arptcodes = read_lake(spark, output_path, arptcodes_fldr)

In [41]:
# I94 Quality Checks
has_records(df_i94, i94_fldr)
check_prikey_null(df_i94, i94_fldr, "cic_id")

Check Passed in table 40790529 with record count: immi_fact
Primary Key Null check passed for Table: immi_fact


True

In [42]:
# World Temp Quality Checks
has_records(df_worldtemp, worldtemp_fldr)
check_prikey_null(df_worldtemp, worldtemp_fldr, "date_stamp")

Check Passed in table 8235082 with record count: worldtemp_dim
Primary Key Null check passed for Table: worldtemp_dim


True

In [43]:
# US Demographics Quality Checks
columns = ['city', 'state']
has_records(df_demog, demog_fldr)
check_prikey_null(df_demog, demog_fldr, columns)

Check Passed in table 2891 with record count: demog_dim
Primary Key Null check passed for Table: demog_dim


True

In [44]:
# Airport Codes Quality Checks
has_records(df_arptcodes, arptcodes_fldr)
check_prikey_null(df_arptcodes, arptcodes_fldr, "airport_id")

Check Passed in table 55075 with record count: arptcodes_dim
Primary Key Null check passed for Table: arptcodes_dim


True

##### Analytics Sample:

In [5]:
# Create a simple table that combines Immigration and Demographics data
# Using a slice of data on a small selection of states for the sample

state_cond = """state_code = 'AK' OR
                state_code = 'MA' OR
                state_code = 'PA' OR
                state_code = 'AL' OR
                state_code = 'NY'"""

df_immi_foreign = df_i94.join(df_demog, df_i94.arr_state == df_demog.state_code, 'inner') \
                        .where(state_cond) \
                        .select(df_i94.cic_id, \
                                df_demog.state_code, \
                                df_demog.state, \
                                df_demog.foreign_born) \
                        .sort("state") \
                        .distinct()

In [6]:
df_immi_foreign.createOrReplaceTempView("analytics_sample")

In [7]:
# The following query gets count of New Immigrants and Foreign born by state

sql_qry = """SELECT state,
                    COUNT(cic_id) AS new_immigrant_count,
                    SUM(foreign_born) AS foreign_born_by_state
             FROM analytics_sample
             GROUP BY state
             ORDER BY state"""

df_result = spark.sql(sql_qry)

df_result.toPandas()

,state,new_immigrant_count,foreign_born_by_state
0,Alabama,701974,5230107428
1,Alaska,79189,2633667762
2,Massachusetts,13814332,513692842586
3,New York,48804954,15254125913934
4,Pennsylvania,3001439,124714078220


In [8]:
spark.catalog.dropTempView("analytics_sample")

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

##### Source Data Dictionary

I94 Immigration Data:

| Field    | Type   | Description                                                                                      |
|----------|--------|--------------------------------------------------------------------------------------------------|
| cicid    | double | Record ID                                                                                        |
| i94yr    | double | 4 Digit Year                                                                                     |
| i94mon   | double | Numeric Month                                                                                    |
| i94cit   | double | 3 Digit Code of Birth Country                                                                      |
| i94res   | double | 3 Digit Code for Country of Residence                                                            |
| i94port  | string | Port of Admission Code                                                                           |
| arrdate  | double | Arrival Date in the USA                                                                          |
| i94mode  | double | Mode of Travel Code                                                                              |
| i94addr  | string | State of Arrival                                                                                 |
| depdate  | double | Departure Date from the USA                                                                      |
| i94bir   | double | Age of Respondent in Years                                                                       |
| i94visa  | double | Visa codes collapsed into three categories                                                       |
| count    | double | Used for summary statistics                                                                      |
| dtadfile | string | Character Date Field - Date added to I-94 Files - CIC does not use                               |
| visapost | string | Department of State where where Visa was issued - CIC does not use                               |
| occup    | string | Occupation that will be performed in U.S. - CIC does not use                                     |
| entdepa  | string | Arrival Flag - admitted or paroled into the U.S. - CIC does not use                              |
| entdepd  | string | Departure Flag - Departed, lost I-94 or is deceased - CIC does not use                           |
| entdepu  | string | Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use      |
| matflag  | string | Match flag - Match of arrival and departure records                                              |
| biryear  | double | 4 digit year of birth                                                                            |
| dtaddto  | string | Character Date Field - Date to which admitted to U.S. (allowed to stay until) - CIC does not use |
| gender   | string | Non-immigrant sex                                                                                |
| insnum   | string | INS number                                                                                       |
| airline  | string | Airline used to arrive in U.S.                                                                   |
| admnum   | double | Admission Number                                                                                 |
| fltno    | string | Flight number of Airline used to arrive in U.S.                                                  |
| visatype | string | Class of admission legally admitting the non-immigrant to temporarily stay in U.S.               |

World Temperature Data:

| Field                         | Type      | Description                     |
|-------------------------------|-----------|---------------------------------|
| dt                            | timestamp | Sample Timestamp                |
| AverageTemperature            | double    | Average Temperature             |
| AverageTemperatureUncertainty | double    | Average Temperature Uncertainty |
| City                          | string    | City                            |
| Country                       | string    | Country                         |
| Latitude                      | string    | Latitude with N - S indicator   |
| Longitude                     | string    | Longitude with E - W indicator  |
| Fi
US Demographics Data:

| Field                  | Type    | Description                       |
|------------------------|---------|-----------------------------------|
| City                   | string  | US City                           |
| State                  | string  | US State                          |
| Median Age             | double  | Median Age                        |
| Male Population        | integer | Male Population Count             |
| Female Population      | integer | Female Population Count           |
| Total Population       | integer | Total Population Count            |
| Number of Veterans     | integer | Veteran Count                     |
| Foreign-born           | integer | Foreign born Count                |
| Average Household Size | double  | Average Household Size            |
| State Code             | string  | State Abbreviation                |
| Race                   | string  | Race Identifier                   |
| Count                  | integer | Count of Individuals of Each Race |

Airport Code Data:

| Field        | Type    | Description                                                                                                                                                                                                                                                                                                                                                                    |
|--------------|---------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| ident        | string  | The text identifier used in the OurAirports URL. This will be the ICAO code if available. Otherwise, it will be a local airport code (if no conflict), or if nothing else is available, an internally-generated code starting with the ISO2 country code, followed by a dash and a four-digit number.                                                                          |
| type         | string  | The type of the airport. Allowed values are "closed_airport", "heliport", "large_airport", "medium_airport", "seaplane_base", and "small_airport".                                                                                                                                                                                                                             |
| name         | string  | The official airport name, including "Airport", "Airstrip", etc.                                                                                                                                                                                                                                                                                                               |
| elevation_ft | integer | The airport elevation MSL in feet (not metres).                                                                                                                                                                                                                                                                                                                                |
| continent    | string  | The code for the continent where the airport is (primarily) located. Allowed values are "AF" (Africa), "AN" (Antarctica), "AS" (Asia), "EU" (Europe), "NA" (North America), "OC" (Oceania), or "SA" (South America).                                                                                                                                                           |
| iso_country  | string  | The two-character ISO 3166:1-alpha2 code for the country where the airport is (primarily) located. A handful of unofficial, non-ISO codes are also in use, such as "XK" for Kosovo. Points to the code column in countries.csv.                                                                                                                                                |
| iso_region   | string  | An alphanumeric code for the high-level administrative subdivision of a country where the airport is primarily located (e.g. province, governorate), prefixed by the ISO2 country code and a hyphen. OurAirports uses ISO 3166:2 codes whenever possible, preferring higher administrative levels, but also includes some custom codes. See the documentation for regions.csv. |
| municipality | string  | The primary municipality that the airport serves (when available). Note that this is not necessarily the municipality where the airport is physically located.                                                                                                                                                                                                                 |
| gps_code     | string  | The code that an aviation GPS database (such as Jeppesen's or Garmin's) would normally use for the airport. This will always be the ICAO code if one exists. Note that, unlike the ident column, this is not guaranteed to be globally unique.                                                                                                                                 |
| iata_code    | string  | he three-letter IATA code for the airport (if it has one).                                                                                                                                                                                                                                                                                                                     |
| local_code   | string  | The local country code for the airport, if different from the gps_code and iata_code fields (used mainly for US airports).                                                                                                                                                                                                                                                     |
| coordinates  | string  | Malformed Airport Coordinates.  The Longitude is listed before the Latitude.                                                                                                                                                                                                                                                                                                   |

##### Data Lake Dictionary:

immi_fact table:

| Field              | Type    | Description                                                                                      |
|--------------------|---------|--------------------------------------------------------------------------------------------------|
| cic_id             | integer | Record ID                                                                                        |
| i94_year           | integer | 4 Digit Year                                                                                     |
| i94_month          | integer | Numeric Month                                                                                    |
| birth_country_code | integer | 3 Digit Code of Birth Country                                                                    |
| res_country_code   | integer | 3 Digit Code for Country of Residence                                                            |
| admit_port         | string  | Port of Admission Code                                                                           |
| arrival_date       | date    | Arrival Date in the USA                                                                          |
| trans_mode         | integer | Mode of Travel Code                                                                              |
| arr_state          | string  | State of Arrival                                                                                 |
| departure_date     | date    | Departure Date from the USA                                                                      |
| immigrant_age      | integer | Age of Respondent in Years                                                                       |
| visa_code          | integer | Visa codes collapsed into three categories                                                       |
| summary_stat       | integer | Used for summary statistics                                                                      |
| file_add_date      | date    | Character Date Field - Date added to I-94 Files - CIC does not use                               |
| visa_post          | integer | Department of State where where Visa was issued - CIC does not use                               |
| us_occupation      | string  | Occupation that will be performed in U.S. - CIC does not use                                     |
| arr_flag           | string  | Arrival Flag - admitted or paroled into the U.S. - CIC does not use                              |
| dep_flag           | string  | Departure Flag - Departed, lost I-94 or is deceased - CIC does not use                           |
| upd_flag           | string  | Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use      |
| match_flag         | string  | Match flag - Match of arrival and departure records                                              |
| birth_year         | integer | 4 digit year of birth                                                                            |
| addmission_date    | date    | Character Date Field - Date to which admitted to U.S. (allowed to stay until) - CIC does not use |
| gender             | string  | Non-immigrant sex                                                                                |
| ins_num            | string  | INS number                                                                                       |
| airline_code       | string  | Airline used to arrive in U.S.                                                                   |
| admit_num          | integer | Admission Number                                                                                 |
| flight_num         | string  | Flight number of Airline used to arrive in U.S.                                                  |
| visa_type          | string  | Class of admission legally admitting the non-immigrant to temporarily stay in U.S.               |

worldtemp_dim table:

| Field           | Type   | Description                     |
|-----------------|--------|---------------------------------|
| date_stamp      | date   | Sample Date                     |
| avg_temp        | float  | Average Temperature             |
| avg_temp_uncert | float  | Average Temperature Uncertainty |
| city            | string | City                            |
| country         | string | Country                         |
| latitude        | string | Latitude with N - S indicator   |
| longitude       | string | Longitude with E - W indicator  |

demog_dim table:

| Field              | Type    | Description                       |
|--------------------|---------|-----------------------------------|
| city               | string  | US City                           |
| state              | string  | US State                          |
| median_age         | double  | Median Age                        |
| male_population    | integer | Male Population Count             |
| female_population  | integer | Female Population Count           |
| total_population   | integer | Total Population Count            |
| num_of_veterans    | integer | Veteran Count                     |
| foreign_born       | integer | Foreign born Count                |
| avg_household_size | double  | Average Household Size            |
| state_code         | string  | State Abbreviation                |
| race               | string  | Race Identifier                   |
| count              | integer | Count of Individuals of Each Race |

arptcodes_dim table:

| Field           | Type    | Description                                                                                                                                                                                                                                                                                                                                                                    |
|-----------------|---------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| airport_id      | string  | The text identifier used in the OurAirports URL. This will be the ICAO code if available. Otherwise, it will be a local airport code (if no conflict), or if nothing else is available, an internally-generated code starting with the ISO2 country code, followed by a dash and a four-digit number.                                                                          |
| airport_type    | string  | The type of the airport. Allowed values are "closed_airport", "heliport", "large_airport", "medium_airport", "seaplane_base", and "small_airport".                                                                                                                                                                                                                             |
| airport_name    | string  | The official airport name, including "Airport", "Airstrip", etc.                                                                                                                                                                                                                                                                                                               |
| elevation_ft    | integer | The airport elevation MSL in feet (not metres).                                                                                                                                                                                                                                                                                                                                |
| continent_abbrv | string  | The code for the continent where the airport is (primarily) located. Allowed values are "AF" (Africa), "AN" (Antarctica), "AS" (Asia), "EU" (Europe), "NA" (North America), "OC" (Oceania), or "SA" (South America).                                                                                                                                                           |
| country_code    | string  | The two-character ISO 3166:1-alpha2 code for the country where the airport is (primarily) located. A handful of unofficial, non-ISO codes are also in use, such as "XK" for Kosovo. Points to the code column in countries.csv.                                                                                                                                                |
| region_code     | string  | An alphanumeric code for the high-level administrative subdivision of a country where the airport is primarily located (e.g. province, governorate), prefixed by the ISO2 country code and a hyphen. OurAirports uses ISO 3166:2 codes whenever possible, preferring higher administrative levels, but also includes some custom codes. See the documentation for regions.csv. |
| municipality    | string  | The primary municipality that the airport serves (when available). Note that this is not necessarily the municipality where the airport is physically located.                                                                                                                                                                                                                 |
| gps_code        | string  | The code that an aviation GPS database (such as Jeppesen's or Garmin's) would normally use for the airport. This will always be the ICAO code if one exists. Note that, unlike the ident column, this is not guaranteed to be globally unique.                                                                                                                                 |
| iata_code       | string  | he three-letter IATA code for the airport (if it has one).                                                                                                                                                                                                                                                                                                                     |
| local_code      | string  | The local country code for the airport, if different from the gps_code and iata_code fields (used mainly for US airports).                                                                                                                                                                                                                                                     |
| latitude        | double  | Latitude of the airport in decimal format                                                                                                                                                                                                                                                                                                                                      |
| longitude       | double  | Longitude of the airport in decimal format                                                                                                                                                                                                                                                                                                                                     |

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.

For this project I used Apache Spark.  Getting the data extracted, transformed and loaded is heavily compute intensive with a large dataset.  With Spark, I was able to pull in the data from a couple of different sources, do some data cleaning and correct an error with coordinate data and load the data into a data lake.  Apache Spark is a scalable solution for datasets of many different sizes and is very useful in the big data ecosystem.  Running Apache Spark on AWS EMR, scaling to meet new demands is very quick.

* Propose how often the data should be updated and why.  
  
  
    - The I94 Immigration Data should be updated once a month as that is the source data appears to be updated by looking at the file name.
    - World Temperature Data should be updated once a day as the data uses an interval of 1 day.
    - The Demographics Data should be updated once every 10 years as the information is from the U.S. Census Bureau and comes from Census data, which is collected once every 10 years.
    - The Airport Code Data should be updated every day in the early AM as the dataset is updated nightly.  

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 
    If the data was increased by 100, a solution like AWS EMR and AWS S3 can be scaled to handle the increased dataset.  
  
  
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
  
    To automate the population of a dashboard daily by 7 AM, a workflow orchestrator such as Apache Airflow can be used.  For datasets on different timelines, it would be possible to create seperate Directed Acyclical Graphs to handle different datasets that refresh on different timelines.  With Directed Acyclical Graphs, the automation of extraction, cleaning, quality checking and loading can all be done effectively.  
  
  
 * The database needed to be accessed by 100+ people.
  
    If the data needed to be accessed by 100 or more people, AWS S3 would be adaquate for the job as would AWS Redshift.  For in-house systems hosted on a large corporate data farm, you could also scale your systems and use mirroring in instances where a single blade on a rack is not adaquate to handle the workload.  
     